In [4]:
# Short script to test the projected gradient method against other
# algorithms.
using Distributions
using LowRankApprox
include("../code/datasim.jl");
include("../code/likelihood.jl");
include("../code/mixEM.jl");
include("../code/mixGD.jl");
include("../code/mixSQP.jl");

# Initialize the sequence of pseudorandom numbers.
srand(1);

# Generate a data set with n = 50,000.
z = normtmixdatasim(round(Int,1000));

# Compute the 50,000 x 20 likelihood matrix.
sd = autoselectmixsd(z,nv = 1000);
L  = normlikmatrix(z,sd = sd);

In [11]:
# Initialize the sequence of pseudorandom numbers.
srand(2018);

# Generate a data set with n = 50,000.
z = normtmixdatasim(round(Int,1000));

# Compute the 50,000 x 20 likelihood matrix.
sd = autoselectmixsd(z,nv = 1000);
L  = normlikmatrix(z,sd = sd);

In [12]:
# Run the mix-SQP algorithm.
@time outsqp = mixSQP(L,lowrank = "none", eps = 1e-8, maxqpiter = 20, verbose = false, nullprior = 0);
@time outsqp2 = mixSQP(L,lowrank = "qr", eps = 1e-8, maxqpiter = 20, verbose = false, nullprior = 0);

# Run the EM algorithm.
@time xem, fem = mixEM(L,maxiter = 200,tol = 1e-6);

# Run the projected gradient descent method.
@time xpgd, fpgd = mixGD(L,maxiter = 6000, alpha = 10, tol = 1e-8);

# Compare the quality of the solutions.
@printf "Objective at SQP  solution: %0.12e\n" mixobjective(L,outsqp["x"])
@printf "Objective at SQP2 solution: %0.12e\n" mixobjective(L,outsqp2["x"])
@printf "Objective at EM   solution:  %0.12e\n" mixobjective(L,xem)
@printf "Objective at PGD  solution: %0.12e\n" mixobjective(L,xpgd)

  2.077090 seconds (46.86 k allocations: 1.375 GiB, 60.50% gc time)
  1.728100 seconds (242.56 k allocations: 967.896 MiB, 9.19% gc time)
  4.002609 seconds (407.77 k allocations: 4.490 GiB, 13.05% gc time)
  6.584784 seconds (1.60 M allocations: 1.090 GiB, 1.71% gc time)
Objective at SQP  solution: 1.984644174655e+02
Objective at SQP2 solution: 1.984644174655e+02
Objective at EM   solution:  1.996963098746e+02
Objective at PGD  solution: 1.985189779534e+02
